In [1]:
import pandas as pd
from prophet import Prophet

/Users/johanbjerkem/Library/CloudStorage/OneDrive-NTNU/Documents/NTNU/5klassehost/ads/TDT4259/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [2]:
# Load data
data_path = "data_NO2_boosted.csv"
df = pd.read_csv(data_path)

# Convert to datetime and remove timezone
df['date'] = pd.to_datetime(df['date']).dt.tz_localize(None)

# Rename columns
df = df.rename(columns={'spot_price': 'y', 'date': 'ds'})

# Display the updated DataFrame
df.head()


,ds,y,krs_temp_2m,stv_temp_2m,gas_price
0,2015-12-31 23:00:00,16.39,6.3,7.3,2.28
1,2016-01-01 00:00:00,16.04,6.1,6.9,2.28
2,2016-01-01 01:00:00,15.74,6.3,7.0,2.28
3,2016-01-01 02:00:00,15.57,6.5,7.4,2.28
4,2016-01-01 03:00:00,15.47,6.7,8.0,2.28


In [3]:
# Initialize variables for the rolling forecast
forecast_results = []  # List to store predictions
total_hours = 75  # Example number of hours to predict; adjust as needed
forecast_horizon = 1  # Set forecast horizon to 1 hour

# Start rolling forecast
for i in range(total_hours):
    print(f"Hour {i+1} of {total_hours}")

    # Define current train set excluding the last i hours
    df_copy = df.copy()

    # Calculate the starting index to keep exactly 364 days of data (364*24 hours)
    start_index = -(364 * 24 + i) if i < 364 * 24 else -i
    train_data = df_copy.iloc[start_index:-i] if i > 0 else df_copy

    # Initialize Prophet model and add extra regressors
    m = Prophet(daily_seasonality="auto", weekly_seasonality="auto", yearly_seasonality="auto")
    m.add_regressor('stv_temp_2m')
    m.add_regressor('gas_price')

    # Fit the model on the latest training data
    m.fit(train_data)

    # Create a future dataframe for the next hour
    future = m.make_future_dataframe(periods=forecast_horizon, freq='H')

    # Add regressor data for the next hour
    future['stv_temp_2m'] = df['stv_temp_2m'].iloc[:len(future)].values
    future['gas_price'] = df['gas_price'].iloc[:len(future)].values

    # Make a one-hour prediction
    forecast = m.predict(future)

    # Store the prediction for this hour only
    forecast_1h = forecast[['ds', 'yhat']].iloc[-forecast_horizon:]
    forecast_results.append(forecast_1h)

    # Append the actual data for this hour to the training set
    if i < len(df) - 1:
        df_copy = df_copy.append(df.iloc[i + 1])

Hour 1 of 75


09:57:17 - cmdstanpy - INFO - Chain [1] start processing
09:57:43 - cmdstanpy - INFO - Chain [1] done processing
/Users/johanbjerkem/Library/CloudStorage/OneDrive-NTNU/Documents/NTNU/5klassehost/ads/TDT4259/.venv/lib/python3.12/site-packages/prophet/forecaster.py:1854: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  dates = pd.date_range(


ValueError: Length of values (23666) does not match length of index (23667)

In [ ]:
# Combine all results into a single DataFrame
forecast_results_df = pd.concat(forecast_results, ignore_index=True)

# Display the results
print(forecast_results_df)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

# Rename columns in forecast_results_df to prepare for merge
forecast_results_df.columns = ['ds', 'yhat']

# Merge forecast results with actual values from df based on the timestamp 'ds'
# Selecting only the necessary columns for alignment
merged_df = pd.merge(forecast_results_df, df[['ds', 'y']], on='ds', how='inner')

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(merged_df['y'], merged_df['yhat'])
print(f"Mean Absolute Error (MAE): {mae}")

# Calculate Root Mean Squared Error (RMSE)
rmse = np.sqrt(mean_squared_error(merged_df['y'], merged_df['yhat']))
print(f"Root Mean Squared Error (RMSE): {rmse}")


Train with 364 days (date and temp*2 stv krs and gasprice):Mean Absolute Error (MAE): 4.640416961052217
Root Mean Squared Error (RMSE): 6.426197707555815

Train with 364 days (date and demand): Mean Absolute Error (MAE): 3.5591405415881954
Root Mean Squared Error (RMSE): 5.0292826198025

In [ ]:
import matplotlib.pyplot as plt

# Plot the forecasted values
plt.figure(figsize=(12, 6))
plt.plot(forecast['ds'], forecast['yhat'], color='blue', label='Predicted Spot Price')

# Plot the actual test data
plt.plot(df_test['datetime_utc'], df_test['spot_price'], color='red', label='Actual Test Data')

# Add labels and legend
plt.xlabel('Date')
plt.ylabel('Spot Price')
plt.title("Prophet Forecast vs. Actual Test Data")
plt.legend()
plt.show()


In [ ]:
from sklearn.metrics import mean_squared_error
import numpy as np

# Extract the forecasted values for the test period
forecast_test = forecast.iloc[-len(df_test):]  # Select the last part of the forecast that matches the test period

# Calculate MSE and RMSE
mse = mean_squared_error(df_test['spot_price'], forecast_test['yhat'])
rmse = np.sqrt(mse)

print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
